### 41. Calcular el sentimiento promedio de cada categoría en base al promedio ponderado del sentimiento de sus géneros. Si por ej una categoría tiene género A y B, A tiene asociado 1 y B asociado -1, si A tiene el 75% y B el 25%, la categoría tiene un sentimiento promedio de 0.5 ⭐⭐